In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import pickle


In [3]:
with open("model.pkl", "rb") as f:
    model = pickle.load(f)


In [4]:
df = pd.read_csv('data_test.csv')

In [5]:
columns_dict = {
    'one_hot': ['country','gender'], 
    'without_X': ['customer_id']
    }

class Processing():
    def __init__(self, df: pd.DataFrame) -> None:
        self.df = df

    def one_hot(self, columns_dict: dict):

        self.df = pd.get_dummies(self.df, columns=columns_dict['one_hot'], drop_first=False)

        return self.df
    
    def feature_label(self, columns_dict: dict) -> pd.DataFrame:

        X = self.df.drop(columns = columns_dict['without_X'])

        return X


    def scale(self, X) -> pd.DataFrame:

        sc = StandardScaler()
        X_sc = sc.fit_transform(X)

        return X_sc

load_df = Processing(df = df)
df = load_df.one_hot(columns_dict=columns_dict)
X = load_df.feature_label(columns_dict=columns_dict)
X_sc = load_df.scale(X)

In [6]:
y_pred = model.predict(X_sc)
y_pred_proba = model.predict_proba(X_sc)

list_proba = list()
for i in range(0, len(y_pred_proba)):
        if y_pred_proba[i][0] > y_pred_proba[i][1]:
            list_proba.append(y_pred_proba[i][0])
        else:
            list_proba.append(y_pred_proba[i][1])

df['y_pred'] = y_pred
df['y_proba'] = list_proba
df.head(20)

,customer_id,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,country_France,country_Germany,country_Spain,gender_Female,gender_Male,y_pred,y_proba
0,15615753,597,35,8,131101.04,1,1,1,192852.67,0,1,0,1,0,0,0.753520
1,15654700,523,40,2,102967.41,1,1,0,128702.10,1,0,0,1,0,0,0.777166
2,15633877,706,42,8,95386.82,1,1,1,75732.25,0,0,1,1,0,0,0.797372
3,15745623,788,32,4,112079.58,1,0,0,89368.59,1,0,0,0,1,0,0.912010
4,15765902,706,38,5,163034.82,2,1,1,135662.17,0,1,0,0,1,0,0.954239
5,15804142,670,57,3,175575.95,2,1,0,99061.75,0,0,1,1,0,1,0.824182
6,15697028,590,34,0,65812.35,2,0,1,160346.30,0,0,1,0,1,0,0.967371
7,15722583,636,29,6,157576.47,2,1,1,101102.39,0,0,1,1,0,0,0.955111
8,15760446,598,64,9,0.00,1,0,1,13181.37,1,0,0,1,0,0,0.603408
9,15588944,456,63,1,165350.61,2,0,0,140758.07,1,0,0,1,0,1,0.797045


In [7]:
score = df[df.customer_id == 15615753]['y_proba']
score.iloc[0]

0.7535196255131082